# Aujourd'hui on se fait rouler par les mecs de l'ENS


https://challengedata.ens.fr/en/challenge/39/prediction_of_transaction_claims_status.html


Ici, c'est le notebook dédié à la gestion de l'imbalanced des classes. On ne touche pas au `test set`.

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python

#load packages
import sys #access to system parameters https://docs.python.org/3/library/sys.html
print("Python version: {}". format(sys.version))

import pandas as pd #collection of functions for data processing and analysis modeled after R dataframes with SQL like features
print("pandas version: {}". format(pd.__version__))

import matplotlib #collection of functions for scientific and publication-ready visualization
print("matplotlib version: {}". format(matplotlib.__version__))

import numpy as np #foundational package for scientific computing
print("NumPy version: {}". format(np.__version__))

import scipy as sp #collection of functions for scientific computing and advance mathematics
print("SciPy version: {}". format(sp.__version__)) 

import IPython
from IPython import display #pretty printing of dataframes in Jupyter notebook
print("IPython version: {}". format(IPython.__version__)) 

import sklearn #collection of machine learning algorithms
print("scikit-learn version: {}". format(sklearn.__version__))

import seaborn as sns
print("seaborn version: {}". format(sns.__version__))

#misc libraries
import time
import random as rnd
import os, gc

#ignore warnings
import warnings
warnings.filterwarnings('ignore')
print('-'*25)

Python version: 3.6.3 |Anaconda custom (64-bit)| (default, Oct 15 2017, 03:27:45) [MSC v.1900 64 bit (AMD64)]
pandas version: 0.22.0
matplotlib version: 2.1.2
NumPy version: 1.12.1
SciPy version: 1.0.0
IPython version: 6.2.1
scikit-learn version: 0.19.1
seaborn version: 0.8.1
-------------------------


Moi j'ai ça:

Python version: 3.6.3 |Anaconda custom (64-bit)| (default, Oct 15 2017, 03:27:45) [MSC v.1900 64 bit (AMD64)]

pandas version: 0.22.0

matplotlib version: 2.1.2

NumPy version: 1.12.1

SciPy version: 1.0.0

IPython version: 6.2.1

scikit-learn version: 0.19.1

seaborn version: 0.8.1


In [2]:
# Run "! pip install jyquickhelper" dans une cellule si ca ne marche pas la commande suivante
from jyquickhelper import add_notebook_menu
add_notebook_menu()

In [3]:
# To plot pretty figures
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
plt.rcParams['axes.labelsize'] = 14
plt.rcParams['xtick.labelsize'] = 12
plt.rcParams['ytick.labelsize'] = 12

## Chargement des données

In [4]:
# Where to save the figures
PROJECT_ROOT_DIR = "."
DATA_PROCESSED = os.path.join(PROJECT_ROOT_DIR, "data_processed")

def load_data(file,data_path=DATA_PROCESSED, sep=','):
    csv_path = os.path.join(data_path, file)
    return pd.read_csv(csv_path, sep)

train_df = load_data(file = "train_notebook_3.csv", sep=';');
test_df = load_data(file = "test_notebook_3.csv", sep=';');


## Analyse des types de données

In [5]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 46 columns):
SHIPPING_MODE                 100000 non-null object
SHIPPING_PRICE                100000 non-null int64
CARD_PAYMENT                  100000 non-null int64
COUPON_PAYMENT                100000 non-null int64
RSP_PAYMENT                   100000 non-null int64
WALLET_PAYMENT                100000 non-null int64
PRICECLUB_STATUS              100000 non-null int64
REGISTRATION_DATE             100000 non-null int64
PURCHASE_COUNT                100000 non-null int64
BUYER_BIRTHDAY_DATE           100000 non-null float64
BUYING_DATE                   100000 non-null int64
SELLER_SCORE_COUNT            100000 non-null int64
SELLER_SCORE_AVERAGE          100000 non-null float64
SELLER_COUNTRY                100000 non-null object
PRODUCT_TYPE                  100000 non-null object
PRODUCT_FAMILY                100000 non-null object
ITEM_PRICE                    100000 non-null int6

## Plan d'action

Comme je suis newbie dans l'imbalanced, on va procéder par étape:

1. Test de l'oversampling avec SMOTE puis ADASYN

2. Combinaison de l'over et under sampling

3. Ensemble of samplers

### Test de l'oversampling avec SMOTE puis ADASYN

While the `RandomOverSampler` is over-sampling by duplicating some of the original samples of the minority class, `SMOTE` and `ADASYN` generate new samples in by interpolation. 

However, the samples used to interpolate/generate new synthetic samples differ. In fact, `ADASYN` focuses on generating samples next to the original samples which are wrongly classified using a `k-Nearest Neighbors` classifier while the basic implementation of `SMOTE` will not make any distinction between easy and hard samples to be classified using the nearest neighbors rule. 

Therefore, the decision function found during training will be different among the algorithms.

In [6]:
from imblearn.over_sampling import SMOTE, ADASYN

In [7]:
from sklearn.preprocessing import LabelEncoder

In [8]:
y=train_df["CLAIM_TYPE"]

X=train_df.drop("CLAIM_TYPE", axis=1, inplace=False)

In [9]:
le_SHIPPING_MODE = LabelEncoder()
X.SHIPPING_MODE=le_SHIPPING_MODE.fit_transform(X.SHIPPING_MODE)

le_SELLER_COUNTRY = LabelEncoder()
X.SELLER_COUNTRY=le_SELLER_COUNTRY.fit_transform(X.SELLER_COUNTRY)

le_PRODUCT_TYPE = LabelEncoder()
X.PRODUCT_TYPE=le_PRODUCT_TYPE.fit_transform(X.PRODUCT_TYPE)

le_PRODUCT_FAMILY = LabelEncoder()
X.PRODUCT_FAMILY=le_PRODUCT_FAMILY.fit_transform(X.PRODUCT_FAMILY)

In [10]:
X.shape

(100000, 45)

In [11]:
y.value_counts()

-                           49977
NOT_RECEIVED                14808
SELLER_CANCEL_POSTERIORI    13782
WITHDRAWAL                   7001
DAMAGED                      5922
DIFFERENT                    4279
UNDEFINED                    4066
FAKE                          165
Name: CLAIM_TYPE, dtype: int64

In [12]:
X_resampled, y_resampled = SMOTE().fit_sample(X, y)

In [13]:
X_resampled = pd.DataFrame(X_resampled, columns=X.columns)
y_resampled = pd.Series(y_resampled, name="CLAIM_TYPE")

In [14]:
X_resampled.shape

(399816, 45)

In [15]:
y_resampled.value_counts()

SELLER_CANCEL_POSTERIORI    49977
WITHDRAWAL                  49977
DIFFERENT                   49977
UNDEFINED                   49977
-                           49977
NOT_RECEIVED                49977
DAMAGED                     49977
FAKE                        49977
Name: CLAIM_TYPE, dtype: int64

In [16]:
X_resampled.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 399816 entries, 0 to 399815
Data columns (total 45 columns):
SHIPPING_MODE                 399816 non-null float64
SHIPPING_PRICE                399816 non-null float64
CARD_PAYMENT                  399816 non-null float64
COUPON_PAYMENT                399816 non-null float64
RSP_PAYMENT                   399816 non-null float64
WALLET_PAYMENT                399816 non-null float64
PRICECLUB_STATUS              399816 non-null float64
REGISTRATION_DATE             399816 non-null float64
PURCHASE_COUNT                399816 non-null float64
BUYER_BIRTHDAY_DATE           399816 non-null float64
BUYING_DATE                   399816 non-null float64
SELLER_SCORE_COUNT            399816 non-null float64
SELLER_SCORE_AVERAGE          399816 non-null float64
SELLER_COUNTRY                399816 non-null float64
PRODUCT_TYPE                  399816 non-null float64
PRODUCT_FAMILY                399816 non-null float64
ITEM_PRICE               

Magnifique, on a des classes biens equilibrées, mais je n'ai aucune idée si c'est un bon sampling ou pas.

On a qu'a dire que oui, de toute façon, ça fait la baseline de l'oversampling. Maintenant on remet les classes comme il faut.

In [17]:
X_resampled['SHIPPING_MODE']=le_SHIPPING_MODE.inverse_transform(X_resampled['SHIPPING_MODE'].round().astype(int))


X_resampled['SELLER_COUNTRY']=le_SELLER_COUNTRY.inverse_transform(X_resampled.SELLER_COUNTRY.round().astype(int))


X_resampled['PRODUCT_TYPE']=le_PRODUCT_TYPE.inverse_transform(X_resampled.PRODUCT_TYPE.round().astype(int))


X_resampled['PRODUCT_FAMILY']=le_PRODUCT_FAMILY.inverse_transform(X_resampled.PRODUCT_FAMILY.round().astype(int))

In [18]:
X_resampled["CLAIM_TYPE"]=y_resampled

In [19]:
X_resampled.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 399816 entries, 0 to 399815
Data columns (total 46 columns):
SHIPPING_MODE                 399816 non-null object
SHIPPING_PRICE                399816 non-null float64
CARD_PAYMENT                  399816 non-null float64
COUPON_PAYMENT                399816 non-null float64
RSP_PAYMENT                   399816 non-null float64
WALLET_PAYMENT                399816 non-null float64
PRICECLUB_STATUS              399816 non-null float64
REGISTRATION_DATE             399816 non-null float64
PURCHASE_COUNT                399816 non-null float64
BUYER_BIRTHDAY_DATE           399816 non-null float64
BUYING_DATE                   399816 non-null float64
SELLER_SCORE_COUNT            399816 non-null float64
SELLER_SCORE_AVERAGE          399816 non-null float64
SELLER_COUNTRY                399816 non-null object
PRODUCT_TYPE                  399816 non-null object
PRODUCT_FAMILY                399816 non-null object
ITEM_PRICE                   

On inscrit nos résultats

In [20]:
filename_train_balanced = DATA_PROCESSED+"/train_balanced.csv"
filename_train = DATA_PROCESSED+"/train.csv"
filename_test = DATA_PROCESSED+"/test.csv"

try:
    os.remove(filename_train)
    os.remove(filename_test)
    os.remove(filename_train_balanced)
except:
    pass;

X_resampled.to_csv(filename_train_balanced, index=False, sep=";")
train_df.to_csv(filename_train, index=False, sep=";")
test_df.to_csv(filename_test, index=False, sep=";")